In [1]:
import requests
import folium
import json
import pystac
import rioxarray
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib as mpl
import geopandas as gpd
import branca
from branca.colormap import LinearColormap
from folium import plugins

In [2]:
resolution = 80

In [3]:
m = folium.Map(
    location=(40.76, -111.89),
    tiles="Stamen Terrain", #"cartodbpositron"
    zoom_start=5,)


minimap = plugins.MiniMap(toggle_display=True)
m.add_child(minimap)

interannual_medians = folium.FeatureGroup(name='2015-2022 Median Snowmelt Runoff Onset Date',show=True)
interannual_std = folium.FeatureGroup(name='2015-2022 Standard Deviation of Snowmelt Runoff Onset Date',show=False)
interannual_corr = folium.FeatureGroup(name='2015-2022 Snowmelt Runoff Onset Date "Trend" Strength',show=False)
year_2015 = folium.FeatureGroup(name='2015 Snowmelt Runoff Onset Date',show=False)
year_2016 = folium.FeatureGroup(name='2016 Snowmelt Runoff Onset Date',show=False)
year_2017 = folium.FeatureGroup(name='2017 Snowmelt Runoff Onset Date',show=False)
year_2018 = folium.FeatureGroup(name='2018 Snowmelt Runoff Onset Date',show=False)
year_2019 = folium.FeatureGroup(name='2019 Snowmelt Runoff Onset Date',show=False)
year_2020 = folium.FeatureGroup(name='2020 Snowmelt Runoff Onset Date',show=False)
year_2021 = folium.FeatureGroup(name='2021 Snowmelt Runoff Onset Date',show=False)
year_2022 = folium.FeatureGroup(name='2022 Snowmelt Runoff Onset Date',show=False)
layer_all_years = [interannual_medians,interannual_std,interannual_corr,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022]
years = ['allyears_median','allyears_std','allyears_corr_strength','2015_median','2016_median','2017_median','2018_median','2019_median','2020_median','2021_median','2022_median']

#tiler = "https://api.cogeo.xyz/cog/tiles/{z}/{x}/{y}"
tiler = 'https://titiler.xyz/cog/tiles/{z}/{x}/{y}'
    
UTM_zones = [10,11,12,13]
for layer_year, year in zip(layer_all_years,years):
    
    if year == 'allyears_std': 
        expression = f"rescale=0,60&colormap_name=reds&nodata=nan"
    elif year == 'allyears_corr_strength':
        expression = f"rescale=-1,1&colormap_name=rdbu&nodata=nan"
    else:
        expression = f"rescale=0,220&colormap_name=viridis&nodata=-32768"
        
    for zone in UTM_zones:
        tif = f"https://raw.githubusercontent.com/egagli/view_sar_snowmelt_timing_map/main/vrts/vrt_{year}_utm{zone}.vrt"
        virtual_tiles = f"{tiler}?url={tif}&{expression}"
        layer_year.add_child(folium.TileLayer(tiles=virtual_tiles,min_zoom=2,max_zoom=12,overlay=True,opacity=1,name=f'{year} Snowmelt Runoff Onset', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))
    

for layer_year in layer_all_years:
    layer_year.add_to(m)


mgrs_squares = 'https://raw.githubusercontent.com/scottyhq/mgrs/main/MGRS_LAND.geojson'
mystyle = lambda x: {'color': 'black','fillOpacity':0,'opacity':0.5}
gjson = folium.GeoJson(mgrs_squares, 
                       embed=False,
                       show=False,
                       style_function=mystyle,
                       #fill_opacity=0.3,
                       name="MGRS Tiles").add_to(m)

folium.features.GeoJsonTooltip(fields=['tile'], labels=False).add_to(gjson)

folium.LayerControl().add_to(m)

cm = branca.colormap.linear.viridis.scale(0,220)
cm.add_to(m)

cm = branca.colormap.linear.Reds_06.scale(0,60)
cm.add_to(m)

cm = branca.colormap.linear.RdBu_06.scale(-1,1)
cm.add_to(m)

m

In [4]:
m.save("index.html")